In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms

import torch.nn.parallel
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
batchSize = 64 
imageSize = 64 

In [7]:
transform = transforms.Compose([transforms.Resize(imageSize), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                transforms.ToTensor()])

In [8]:
transform

Compose(
    Resize(size=64, interpolation=PIL.Image.BILINEAR)
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ToTensor()
)

In [9]:
dataset = dset.CIFAR10(root = './data', download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2)

26.5%

KeyboardInterrupt: 